### Socratic API

We can also access the NTD Annual Fuel and Energy data using a free API endpoint with _unlimited_ usage, as long as we are using an [application token](https://dev.socrata.com/docs/app-tokens). We can query this data using this endpoint, https://data.transportation.gov/resource/8ehq-7his.json, and we can filter the data using [SoQL clauses](https://dev.socrata.com/docs/queries/). 

In [2]:
# Requires requests, pip install requests
import requests
import json

# Define querying function
def query_ntd(params):
    url = "https://data.transportation.gov/resource/8ehq-7his.json"
    response = requests.get(url, params=params)
    # print(response.url)
    if response.status_code == 200:
        data = response.json()
        print(json.dumps(data, indent=2))
    else:
        print(f"Request failed with status code {response.status_code}")

# Define our SoQL clauses
basic_params = {
    '$select': 'distinct city',
    '$where': 'state="NY"',
    '$order': 'city'
}

# Query API w/ our basic clauses
query_ntd(basic_params)

/Users/lgrassi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[
  {
    "city": "Albany"
  },
  {
    "city": "Binghamton"
  },
  {
    "city": "Brooklyn"
  },
  {
    "city": "Buffalo"
  },
  {
    "city": "Calverton"
  },
  {
    "city": "Carmel"
  },
  {
    "city": "Huntington"
  },
  {
    "city": "Hurley"
  },
  {
    "city": "Ithaca"
  },
  {
    "city": "Jamaica"
  },
  {
    "city": "Kingston"
  },
  {
    "city": "Long Beach"
  },
  {
    "city": "Mineola"
  },
  {
    "city": "New City"
  },
  {
    "city": "New York"
  },
  {
    "city": "Poughkeepsie"
  },
  {
    "city": "Rochester"
  },
  {
    "city": "Spring Valley"
  },
  {
    "city": "Staten Island"
  },
  {
    "city": "Syracuse"
  },
  {
    "city": "White Plains"
  },
  {
    "city": "Yaphank"
  }
]


### Gathering data by UACE codes

To find the different column id's that correspond to each column, visit [this](https://data.transportation.gov/Public-Transit/2022-NTD-Annual-Data-Fuel-and-Energy/8ehq-7his/data) website, click **Export**, then **SODA API**, and all of the column id's will be listed below.

In [3]:
def get_data_about_uace(code, select_vals):
    select_params = ",".join(select_vals)
    params = {
        '$select': select_params,
        '$where': 'uace_code="' + code +'"'
    }
    query_ntd(params)
    
get_data_about_uace('63217', ["uace_code","agency","city","gasoline_gal","diesel_gal","gasoline","diesel"])


[
  {
    "uace_code": "63217",
    "agency": "MTA New York City Transit",
    "city": "Brooklyn",
    "gasoline_gal": "0",
    "diesel_gal": "3697741",
    "gasoline": "0",
    "diesel": "15739148"
  },
  {
    "uace_code": "63217",
    "agency": "MTA New York City Transit",
    "city": "Brooklyn",
    "gasoline_gal": "0",
    "diesel_gal": "23170009",
    "gasoline": "0",
    "diesel": "79941725"
  },
  {
    "uace_code": "63217",
    "agency": "MTA New York City Transit",
    "city": "Brooklyn",
    "gasoline_gal": "3570980",
    "diesel_gal": "19738",
    "gasoline": "28059319",
    "diesel": "242940"
  },
  {
    "uace_code": "63217",
    "agency": "MTA New York City Transit",
    "city": "Brooklyn",
    "gasoline_gal": "0",
    "diesel_gal": "1070814",
    "gasoline": "0",
    "diesel": "3180905"
  },
  {
    "uace_code": "63217",
    "agency": "MTA New York City Transit",
    "city": "Brooklyn",
    "gasoline_gal": "0",
    "diesel_gal": "0",
    "gasoline": "0",
    "diesel": "

### Aggregate data manipulation

Using SoQL allows us to do some powerful data aggregation using `$group` and specific clauses found [here](https://dev.socrata.com/docs/queries/group).

In [5]:
gasoline_info_params = {
    '$select': 'uace_code,SUM(gasoline_gal) as total_gas_gals,SUM(gasoline) as total_gas_miles,AVG(gasoline) as avg_gas_miles',
    '$where': 'uace_code="63217"',
    '$group': 'uace_code'
}

print("\nInformation about gasoline usage in UACE 63217")
query_ntd(gasoline_info_params)

ranked_params = {
    "$select": "uace_code, SUM(gasoline_gal) as total_gas_gals",
    "$group": "uace_code",
    "$order": "SUM(gasoline_gal) DESC",
    "$limit": "10"
}

print("\nTop 10 UACE codes in terms of gallons of gas used")
query_ntd(ranked_params)




Information about gasoline usage in UACE 63217
[
  {
    "uace_code": "63217",
    "total_gas_gals": "7750979",
    "total_gas_miles": "66437808",
    "avg_gas_miles": "1145479.448275862069"
  }
]

Top 10 UACE codes in terms of gallons of gas used
[
  {
    "uace_code": "63217",
    "total_gas_gals": "7750979"
  },
  {
    "uace_code": "51445",
    "total_gas_gals": "5235725"
  },
  {
    "uace_code": "57628",
    "total_gas_gals": "4400701"
  },
  {
    "uace_code": "16264",
    "total_gas_gals": "3134130"
  },
  {
    "uace_code": "56602",
    "total_gas_gals": "3069851"
  },
  {
    "uace_code": "40429",
    "total_gas_gals": "2705990"
  },
  {
    "uace_code": "92242",
    "total_gas_gals": "2421185"
  },
  {
    "uace_code": "80389",
    "total_gas_gals": "1871475"
  },
  {
    "uace_code": "4843",
    "total_gas_gals": "1816826"
  },
  {
    "uace_code": "78904",
    "total_gas_gals": "1658109"
  }
]
